In [541]:
import os,sys
import pandas as pd
from pyperclip import copy
import re
import unicodedata as uc

In [542]:
import json

def read_ipynb(input_ipynb_file):
    # Check if the input file is a Jupyter Notebook
    
    # Open the input Jupyter Notebook file
    notebook = open(input_ipynb_file,encoding="utf-8")
    
    # Read its content in the json format
    notebook_content = json.load(notebook)

    # Only extract the source code snippet from each cell in the input Jupyter Notebook
    source_code_snippets = [cell['source'] for cell in notebook_content['cells']]
    text = ""
    for cell in source_code_snippets:
        for line in cell:
            text+= line
        text+= "\n"

    return text
def ipynb_2_py(input_ipynb_file):
    text = read_ipynb(input_ipynb_file)
    text = re.sub(r'display.*\n',"",text,re.MULTILINE)

    imports = re.search(r"### start_imports ###\n(.*)### end_imports ###",
                        text, re.MULTILINE | re.DOTALL|re.UNICODE).group(1)

    functions = re.findall(r"#### start_function ###(.*?)### end_function ###",
                        text, re.MULTILINE | re.DOTALL)

    functions = re.findall(r"#### start_function ###(.*?)### end_function ###",
                        text, re.MULTILINE | re.DOTALL)
    funcoes = ""
    for function in reversed(functions):
        function_vars = re.search(r"### start_function_vars ###\n(.*)### end_function_vars ###",
                            function, re.MULTILINE | re.DOTALL).group(1)

        dummy = ""

        try:
            function_defined_vars = re.search(r"### start_funcion_defined_vars ###\n(.*)\n### end_funcion_defined_vars ###",
                                function, re.MULTILINE | re.DOTALL).group(1)



            for line in function_defined_vars.split("\n"):
                var,value = line.replace(" ",'').split("=")
                dummy += f"{var} = {value}, "
        except AttributeError:
            pass
        function_defined_vars = dummy

        dummy = ""
        try:
            function_vars = re.search(r"### end_funcion_defined_vars ###\n(.*)\n### end_function_vars ###",
                                function, re.MULTILINE | re.DOTALL).group(1)


            for line in function_vars.split("\n"):
                var,*value = line.replace(" ",'').split("=")
                dummy += f"{var}, "
        except AttributeError:
            pass
        function_vars = dummy
        function_arg = function_vars +  function_defined_vars
        function_name,code = re.findall(r"function_name = (.*?)\n.*### start_code ###\n(.*)",
                            function, re.MULTILINE | re.DOTALL)[0]

        code = re.sub(r"\"\"\" ### include_on_script\n(.*)\"\"\"", "\\1", code, 0, re.MULTILINE | re.DOTALL)
        dummy = ""
        for line in code.split("\n"):
            dummy += " "*4 + line + '\n'
        code = dummy
        text = f"{function_name}({function_arg}):\n{dummy}"
        funcoes += text + "\n"
    return f"{imports} \n {funcoes}"

In [555]:
texto = ipynb_2_py("./Previsor_teste3.ipynb")

In [556]:
copy(texto)

AttributeError: 'NoneType' object has no attribute 'group'

In [545]:
functions = re.findall(r"#### start_function ###(.*?)### end_function ###",
                    text, re.MULTILINE | re.DOTALL)

for function in reversed(functions):
    pass
imports = re.search(r"### start_imports ###\n(.*)### end_imports ###",
                    text, re.MULTILINE | re.DOTALL|re.UNICODE).group(1)
function_vars = re.search(r"### start_function_vars ###\n(.*)### end_function_vars ###",
                    function, re.MULTILINE | re.DOTALL).group(1)
dummy = ""

try:
    function_defined_vars = re.search(r"### start_funcion_defined_vars ###\n(.*)\n### end_funcion_defined_vars ###",
                        function, re.MULTILINE | re.DOTALL).group(1)



    for line in function_defined_vars.split("\n"):
        var,value = line.replace(" ",'').split("=")
        dummy += f"{var} = {value}, "
except AttributeError:
    pass
function_defined_vars = dummy

dummy = ""
try:
    function_vars = re.search(r"### end_funcion_defined_vars ###\n(.*)\n### end_function_vars ###",
                        function, re.MULTILINE | re.DOTALL).group(1)


    for line in function_vars.split("\n"):
        var,*value = line.replace(" ",'').split("=")
        dummy += f"{var}, "
except AttributeError:
    pass
function_vars = dummy
function_arg = function_vars +  function_defined_vars
function_name,code = re.findall(r"function_name = (.*?)\n.*### start_code ###\n(.*)",
                    function, re.MULTILINE | re.DOTALL)[0]

code = re.sub(r"\"\"\" ### include_on_script\n(.*)\"\"\"", "\\1", code, 0, re.MULTILINE | re.DOTALL)
dummy = ""
for line in code.split("\n"):
    dummy += " "*4 + line + '\n'
code = dummy
text = f"{function_name}({function_arg}):\n{dummy}"